In [ ]:
%%shell

pip install -q langchain_community langchain_huggingface faiss-cpu gradio openai torch torchvision torchaudio youtokentome pypdf accelerate google-generativeai langchain-google-genai streamlit pillow genai

In [ ]:
%%capture
%%shell

# Install the custom version of NeMo by AI4Bharat
wget https://indic-asr-public.objectstore.e2enetworks.net/ai4b_nemo.zip

unzip -q /content/ai4b_nemo.zip && cd NeMo
bash reinstall.sh

cd ..

In [ ]:
%%capture
%%shell

git clone -q https://github.com/VarunGumma/IndicTransTokenizer
cd IndicTransTokenizer
pip install -q --editable ./
cd ..


In [ ]:
%%capture
%%shell

apt-get install libsndfile1-dev ffmpeg

git clone https://github.com/gokulkarthik/TTS
cd TTS

pip3 install -e .[all]
pip3 install -r requirements.txt

cd ..


In [ ]:
%%capture

# INFO: If you're unable to import these libraries, just rerun this cell again.

import gradio as gr
from torch import cuda, inference_mode
import nemo.collections.asr as nemo_asr
from IndicTransTokenizer import IndicProcessor
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


In [ ]:
DEVICE = "cuda" if cuda.is_available() else "cpu"

print(f"Using device: {DEVICE}")

In [ ]:
import os
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
# Download PM-KISAN offical PDF document
# https://www.pmkisan.gov.in/Documents/RevisedPM-KISANOperationalGuidelines(English).pdf

%%shell

gdown 1qXyBzQ_1uNCocPMAl9Z58UlZGteKl5kg

In [ ]:
pm_kisan_doc = "/content/PM-KISANOperationalGuidelines(English).pdf"

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100
)

loader = PyPDFLoader(pm_kisan_doc)
pages = loader.load_and_split(text_splitter=text_splitter)

pages_chunks = [page.page_content for page in pages]
print(f"Generated {len(pages_chunks)} chunks of {pm_kisan_doc}")

In [ ]:
pages_chunks[8]

In [ ]:
!pip uninstall tensorflow -y
!pip install tensorflow-gpu

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

faiss = FAISS.from_texts(pages_chunks, embeddings)

In [ ]:
# Test query
result = faiss.similarity_search("what are the benefits of PM kisan yojna", k=3)

In [ ]:
# This returns the most relevant doc similar to the query

print(result[0].page_content)

In [ ]:
!pip install genai

In [ ]:
from google.colab import userdata
import genai

genai.configure(api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
!pip install google-generativeai langchain-google-genai streamlit pillow

In [ ]:
import google.generativeai as genai

def get_gemini_output(prompt, temperature=0.2):
    """
    Returns the response from the Gemini AI API.

    Args:
        prompt (str): The input prompt to the API.
        temperature (float): The temperature of the Gemini AI API.

    Returns:
        str: The response content from the Gemini AI API.
    """
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            candidate_count=1,
            stop_sequences=['.'],
            max_output_tokens=200,
            top_p=0.6,
            top_k=5,
            temperature=temperature
        )
    )
    return response.text



In [ ]:
get_gemini_output("who are you")

In [ ]:
ip = IndicProcessor(inference=True)

In [ ]:
%%capture

en2indic_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indictrans2-en-indic-dist-200M", trust_remote_code=True)
en2indic_model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-en-indic-dist-200M", trust_remote_code=True)

In [ ]:
%%capture

indic2en_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indictrans2-indic-en-dist-200M", trust_remote_code=True)
indic2en_model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-en-dist-200M", trust_remote_code=True)

In [ ]:
model_tokenizer_config = {
    "en2indic": {
        "tokenizer": en2indic_tokenizer,
        "model": en2indic_model,
    },
    "indic2en": {
        "tokenizer": indic2en_tokenizer,
        "model": indic2en_model,
    }
}

In [ ]:
def indic_translate(src_lang: str, tgt_lang: str, sents_to_translate: list):

  """
  Uses IndicTrans2-200M distilled model to translate a list of sentences from a src_lang to tgt_lang

  Args:
      src_lang (str): The input sentences language.
      tgt_lang (str): The language in which we want the sentences to be translated to.
      sents_to_translate (list): The list of input sentences.

  Returns:
      outputs: The list of translated sentencesin tgt_lang.
  """

  lang_map = {
    "punjabi": "pan_Guru",
    "bengali": "ben_Beng",
    "malayalam": "mal_Mlym",
    "marathi": "mar_Deva",
    "tamil": "tam_Taml",
    "gujarati": "guj_Gujr",
    "telugu": "tel_Telu",
    "hindi": "hin_Deva",
    "kannada": "kan_Knda",
    "odia": "ory_Orya",
    "english": "eng_Latn"
    }

  src_lang = lang_map[src_lang]
  tgt_lang = lang_map[tgt_lang]

  if src_lang == "eng_Latn":
    tokenizer = model_tokenizer_config["en2indic"]["tokenizer"]
    model = model_tokenizer_config["en2indic"]["model"]

    print(f"Using en2indic, src_lang: {src_lang}, tgt_lang: {tgt_lang}")

  else:
    tokenizer = model_tokenizer_config["indic2en"]["tokenizer"]
    model = model_tokenizer_config["indic2en"]["model"]

    print(f"Using indic2en, src_lang: {src_lang}, tgt_lang: {tgt_lang}")


  batch = ip.preprocess_batch(sents_to_translate, src_lang=src_lang, tgt_lang=tgt_lang, show_progress_bar=False)
  batch = tokenizer(batch, padding="longest", truncation=True, max_length=256, return_tensors="pt")

  with inference_mode():
      print("Generating...")
      outputs = model.generate(**batch, num_beams=5, num_return_sequences=1, max_length=256)

  with tokenizer.as_target_tokenizer():
      outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=True)

  if tgt_lang != "en_Latn":
    print(f"Postprocessing for {tgt_lang}")
    outputs = ip.postprocess_batch(outputs, lang=tgt_lang)


  return outputs


Indic translator test translation

In [ ]:
# Example sentences for translation
test_sentences = ["Hello, how are you?", "This is a test sentence."]

# Call the indic_translate function
translated = indic_translate(src_lang="english", tgt_lang="hindi", sents_to_translate=test_sentences)

# Print the result
print("Translated sentences:", translated)


In [ ]:
def download_ai4b_tts_model(lang: str):

  """
  Download AI4Bharat's TTS models for a given language

  Args:
      lang (str): The language for which we want to download the model.
  """

  lang_map = {
      "odia": "or",
      "hindi": "hi",
      "tamil": "ta",
      "telugu": "te",
      "punjabi": "pa",
      "kannada": "kn",
      "bengali": "bn",
      "marathi": "mr",
      "gujarati": "gu",
      "malayalam": "ml",
  }

  selected_lang = lang_map[lang]

  download_path = f"/content/{selected_lang}.zip"

  if os.path.exists(download_path):
    print(f"IndicTTS Model for {lang} already exists.")

  else:
    !wget https://github.com/AI4Bharat/Indic-TTS/releases/download/v1-checkpoints-release/{selected_lang}.zip
    !mkdir -p /content/models/v1
    !unzip /content/{selected_lang}.zip -d /content/models/v1


In [ ]:
def run_tts(text, tts_lang):

  """
  Convert text to audio using IndicTTS

  Args:
      text (str): The input text, to be converted to audio.
      tts_lang (str): The language in which the text is to be converted to audio.

  """

  lang_map = {
      "odia": "or",
      "hindi": "hi",
      "tamil": "ta",
      "telugu": "te",
      "punjabi": "pa",
      "kannada": "kn",
      "bengali": "bn",
      "marathi": "mr",
      "gujarati": "gu",
      "malayalam": "ml",
  }

  download_ai4b_tts_model(lang=tts_lang)

  tts_lang = lang_map[tts_lang]
  print(f"Lang code: {tts_lang}")


  tts_command = f'python3 -m TTS.bin.synthesize --text "{text}" \
    --model_path /content/models/v1/{tts_lang}/fastpitch/best_model.pth \
    --config_path /content/models/v1/{tts_lang}/fastpitch/config.json \
    --vocoder_path /content/models/v1/{tts_lang}/hifigan/best_model.pth \
    --vocoder_config_path /content/models/v1/{tts_lang}/hifigan/config.json \
    --speakers_file_path /content/models/v1/{tts_lang}/fastpitch/speakers.pth \
    --out_path /content/tts_output.wav \
    --speaker_idx male'

  if DEVICE == "cuda":
    tts_command += " --use_cuda True"
    print(f"Running IndicTTS on GPU")

  else:
    print(f"Running IndicTTS on CPU")

  os.system(tts_command)

Indic translator and TTS model test runing


In [ ]:
# Step 1: Translate the text
test_sentences = ["Hello, how are you?", "This is a test sentence."]
translated_sentences = indic_translate(src_lang="english", tgt_lang="hindi", sents_to_translate=test_sentences)

# Step 2: Generate speech for each translated sentence using the TTS model
for sentence in translated_sentences:
    # Call the TTS function defined in your notebook
    tts_audio_output = download_ai4b_tts_model(lang="hindi")  # Replace with the actual TTS function call if needed

    # Print or play the audio output
    print(f"Generated TTS audio for the translated sentence: {sentence}")
    # Example: Play the audio, save it as a file, or display the audio waveform
    # play_audio(tts_audio_output)  # Replace with actual audio playback code


In [ ]:
!mkdir /content/asr_models

def download_ai4b_asr_model(lang: str):
  """
  Download AI4Bharat's ASR models for a given language

  Args:
      lang (str): The language for which we want to download the model.
  """

  available_langs = {
      "odia": "or",
      "hindi": "hi",
      "tamil": "ta",
      "telugu": "te",
      "punjabi": "pa",
      "kannada": "kn",
      "bengali": "bn",
      "marathi": "mr",
      "gujarati": "gu",
      "malayalam": "ml",
  }

  download_path = f"/content/asr_models/ai4b_indicConformer_{available_langs[lang]}.nemo"
  print(f"Downloaded ASR model path: {download_path}")

  if os.path.exists(download_path):
      print(f"Model for {lang} already exists.")

  elif lang not in available_langs:
      raise ValueError(f"Invalid language code: {lang}")

  else:
    !wget https://objectstore.e2enetworks.net/indic-asr-public/indicConformer/ai4b_indicConformer_{available_langs[lang]}.nemo -O '/content/asr_models/ai4b_indicConformer_{available_langs[lang]}.nemo'

  return download_path

In [ ]:
def transcribe(audio: str, lang: str):

    """
    Uses IndicASR to transcribe the input audio to text, utilizing a Conformer NeMo model trained of Shrutilipi and IndicSuperb by AI4Bharat

    Args:
        audio (str): The input audio's path.
        lang (str): The language of the input audio.

    Returns:
        transcription: The transcription of input audio.
    """

    lang_map = {
      "odia": "or",
      "hindi": "hi",
      "tamil": "ta",
      "telugu": "te",
      "punjabi": "pa",
      "kannada": "kn",
      "bengali": "bn",
      "marathi": "mr",
      "gujarati": "gu",
      "malayalam": "ml",
    }

    download_path = download_ai4b_asr_model(lang=lang)

    asr_model = nemo_asr.models.ASRModel.restore_from(
          download_path, map_location=DEVICE
    )

    transcription = asr_model.transcribe(audio, batch_size=1, language_id=lang_map[lang])[0][0]
    print(f"Transcription: {transcription}")

    return transcription

In [ ]:
def query_vector_db(query):

  """
  Query the FAISS vector DB

  Args:
      query (str): The audio query.

  Returns:
      result (str): Combines top-3 Most similar documents corresponding to the user's query.

  """

  # Combine the top-3 similar documents from the vectorDB
  result = " ".join([result.page_content for result in faiss.similarity_search(query, k=3)])

  return result


In [ ]:
from langchain_core.prompts import PromptTemplate

def process_user_query(user_query, retrieved_doc):

  """
  Uses GPT-3.5-Turbo to extract the part of the retrieved document from vectorDB which is relevant to user's query

  Args:
      user_query (str): The user's question.
      retrieved_doc (str): The information relevant to user's query.

  Returns:
      processed_doc: TThe extracted information from the given document.
  """

  prompt_template = PromptTemplate.from_template(
    "You are a chatbot , which provides information to user based on their queries, \
    the user asks: {user_query}, The information from the related query is: {retrieved_doc}. \
    Now give the output based on the query and relevant information that i provided, written in a structured, well-formatted and concise way. \
    The length of the output should be no more than 70 words, must be in 5 lines."
  )

  prompt = prompt_template.format(user_query=user_query, retrieved_doc=retrieved_doc)

  processed_doc = get_gemini_output(prompt)
  print(processed_doc)

  return processed_doc


In [ ]:
def process_gradio_input(audio, user_lang):

  """
  End-to-end voice assistant pipeline, no. of inputs and outputs are defined in the Gradio interface

  Args:
      audio (str): The audio path.
      lang (str): The user's input language.

  Returns:
      en_to_indic_doc: The first return value. It is the final answer to user's query.
      audio_outfile_path: The second return value. Path to the generated audio having the final answer.

  """

  # Use IndicASR to transcribe the input audio
  print(f"Transcribing...")
  query_transcription = transcribe(audio, lang=user_lang)

  # Convert the Indic text from transcription to English, so that GPT-3.5 can process it
  print(f"Translating indic to en..")
  indic_to_en = indic_translate(src_lang=user_lang, tgt_lang="english", sents_to_translate=[query_transcription])[0]

  # Query the Vector DB to get the relevant document from the query
  print(f"Querying vector db")
  retrieved_doc = query_vector_db(indic_to_en)

  # Extract relevant information from the retrieved document
  print(f"Processing user query")
  processed_doc = process_user_query(user_query=indic_to_en, retrieved_doc=retrieved_doc)

  # Break the document into chunks for faster batch processing
  print(f"Breaking document into chunks..")
  processed_doc_chunks = processed_doc.strip().split(". ")
  processed_doc_chunks = [f"{chunk}." for chunk in processed_doc_chunks if chunk != ""]

  # Translate the the extracted information back to Indic language
  print(f"Translating en to indic..")
  en_to_indic_chunks = indic_translate(src_lang="english", tgt_lang=user_lang, sents_to_translate=processed_doc_chunks)
  en_to_indic_doc = " ".join(en_to_indic_chunks)
  print(f"en_to_indic_doc: {en_to_indic_doc}")

  # Run IndicTTS to generate audio
  print(f"Running TTS to generate audio..")
  run_tts(text=en_to_indic_doc, tts_lang=user_lang)
  print("Finished running TTS")

  audio_outfile_path = "/content/tts_output.wav"


  return en_to_indic_doc, audio_outfile_path


In [ ]:
def launch_gradio_app(show_log=False):

  """
  Launches the Gradio app for the voice assistant pipeline

  Args:
      show_log (bool): Whether to show the pipeline logs or not.

  """

  languages = ["hindi", "odia", "tamil", "telugu", "punjabi", "kannada", "bengali", "marathi", "gujarati", "malayalam"]

  iface = gr.Interface(
      fn=process_gradio_input,
      inputs=[
          gr.Audio(sources=['upload', 'microphone'], type="filepath", show_download_button=True),  # Input audio
          gr.Dropdown(languages, label="Language", value="hindi"),  # Language selection
      ],
      outputs=["text", "audio"],
      allow_flagging="never",
      title="This is made by Srimadhav Varma",
      description="Know about latest farming schemes",
  )

  iface.launch(debug=show_log)


In [ ]:
launch_gradio_app(show_log=False)
